# Analisando Dados com OpenAI e DuckDB

## 0. Preparação

### 0.1 Instalação das bibliotecas necessárias

In [63]:
%%capture
%pip install requests pandas duckdb python-dotenv openai;

### 0.2 Importação das bibliotecas utilizadas

In [64]:
import os
import duckdb
from openai import OpenAI
from dotenv import load_dotenv; 

In [65]:
_ = load_dotenv()
client = OpenAI()


### 0.3 Definição dos templates de prompt

In [66]:
query_prompt_template = """\
<database-context>
{context}
</database-context>

<response-format>
```sql
<<QUERY_HERE>>
```
</response-format>

Crie uma query suportada pelo DuckDB para responder a seguinte pergunta: {question}
"""

In [67]:
response_prompt_template = """\
You are a data analyst responsible for answering questions about the data.

The user asked the following question: 
---
{question}
---

The query to retrieve the data is:
---
{query}
---

This data was retrieved from the database:
---
{data}
---

Answer the question based on the data provided.
"""

### 0.4 Definição de funções utilitárias

In [68]:
def execute_query(query):
    con = duckdb.connect("data/treated/database.duckdb")
    try:
        result = con.execute(query).fetchdf()
        data = result.to_json(orient='records', force_ascii=False)
        status = "success"
    except Exception as e:
        status = "error"
        data = str(e)
    finally:
        con.close()
    return {"status": status, "data": data}

In [69]:
def parse_query(query):
    return query.split("```sql")[1].split("```")[0]

In [70]:
def execute_agent(question, context):
    response = client.responses.create(
        model="gpt-4.1-mini",
        input=query_prompt_template.format(question=question, context=context)
    )

    query = parse_query(response.output_text)
    results = execute_query(query)

    response = client.responses.create(
        model="gpt-4.1-mini",
        input=response_prompt_template.format(data=results["data"], question=question, query=query)
    )

    return response.output_text
    

## 1. Preparação dos dados

### 1.1 Criação do arquivo .duckdb

In [71]:
# Cria o diretório de destino se não existir
os.makedirs('data/treated', exist_ok=True)

# Caminhos dos arquivos
db_path = 'data/treated/database.duckdb'
customers_csv = 'data/raw/customers.csv'
products_csv = 'data/raw/products.csv'
sales_csv = 'data/raw/sales.csv'

# Conecta/cria o banco DuckDB
con = duckdb.connect(db_path)

# Cria as tabelas a partir dos CSVs
con.execute(f"""
    CREATE OR REPLACE TABLE customers AS
    SELECT * FROM read_csv_auto('{customers_csv}');
""")
con.execute(f"""
    CREATE OR REPLACE TABLE products AS
    SELECT * FROM read_csv_auto('{products_csv}');
""")
con.execute(f"""
    CREATE OR REPLACE TABLE sales AS
    SELECT * FROM read_csv_auto('{sales_csv}');
""")

con.close()

### 1.2 Criação do contexto das tabelas do database

In [72]:
import glob

files = glob.glob("schemas/*.yaml")
context = ""
for table_context in files:
    with open(table_context, "r") as f:
        context += f.read()

## 2. Interação com a OpenAI

In [73]:
question = "Quais são os meus principais 10 clientes?"
response = execute_agent(question, context)
print(response)

Seus 10 principais clientes, de acordo com o valor total gasto, são:

1. Krabby Claws – R$ 5.490,41  
2. Granna Goodie – R$ 5.239,44  
3. Astro Pup – R$ 5.008,05  
4. Chirpy McPeep – R$ 4.817,82  
5. Pumba Porker – R$ 4.590,25  
6. George Jetpack – R$ 4.564,34  
7. Slick Coyote – R$ 4.344,52  
8. Peppy Pewt – R$ 4.317,48  
9. Minnie Mousey – R$ 4.072,80  
10. Blaze Runner – R$ 3.947,64  

Esses clientes são os que mais gastaram em suas compras segundo os dados fornecidos.


In [74]:
question = "Quais produtos eu vendi para o cliente Krabby Claws? Qual foi minha receita de cada?"
response = execute_agent(question, context)
print(response)

Para o cliente Krabby Claws, você vendeu os seguintes produtos e obteve a seguinte receita de cada um:

- Giant Rubber Band: R$ 990,49  
- Do-It-Yourself Tornado: R$ 1.364,82  
- Rocket Skates: R$ 1.880,39  
- Dehydrated Boulder: R$ 232,26  
- Earthquake Pills: R$ 526,37  
- Jet-Propelled Unicycle: R$ 496,08  

Se precisar de mais alguma análise ou outro dado, estou à disposição!
